In [ ]:
from hashlib import md5
import requests
import json
import time
import re
import urllib.request as request
from bs4 import BeautifulSoup

In [ ]:
def writeOrderInfo(text):
    fh = open('订单信息.txt', 'a')
    fh.write(text+'\n')
    fh.close()

## 第三方接口

In [ ]:
# 非小号接口：抓取最新整体市场行情（做为风控参照）（单位：USDT）
def getFxhPrice(symbol):
    symbolList = {'BTC_USDT': 'bitcoin', 'ETH_USDT': 'ethereum'}
    url = 'https://www.feixiaohao.com/currencies/'+symbolList[symbol]
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
    requestss = request.Request(url,
                                method='GET',
                                headers=headers)
    data = request.urlopen(requestss).read().decode('utf-8')
    soup = BeautifulSoup(data,'html.parser')
    priceText = soup.find_all('div',attrs={'class':'sub'})[4].get_text()
    priceStr = ""
    for i in re.findall(r"\d+", re.findall(r"\$(.*)\xa0\xa0", priceText)[0]):
        priceStr+=i
    priceInt = int(priceStr)
    return priceInt

## CoinBig官方接口

In [ ]:
# create sign
def addSign(params, secretKey):
    signStr = ""
    for key in params.keys():
        signStr += ("&"+key+"="+str(params[key]))
    signStr = signStr[1:]
    signStr += ("&secret_key="+secretKey)
    sign = md5(signStr.encode("utf8")).hexdigest().upper()
    params['sign'] = sign
    return params

In [ ]:
# 接口1：获取用户资产信息
def getAssetInfo(session, apiKey, secretKey):
    Time = int(time.time() * 1000)
    data = {"apikey": apiKey,
            "time": Time}
    newdata = addSign(data, secretKey)
    redata = session.post('https://www.coinbig.com/api/publics/v1/userinfo', data=newdata).content.decode(encoding='UTF8')
    try:
        redatadict = json.loads(redata)
    except:
        redatadict = 1
        print("获取用户资产信息getAssetInfo接口信息错误！\n\n",redata)
    return redatadict

In [ ]:
# 接口2：获取订单详情
def getOrderInfo(session, apiKey, secretKey, orderId):
    Time = int(time.time() * 1000)
    data = {"apikey": apiKey,
            "order_id": orderId, 
            "time": Time}
    newdata = addSign(data, secretKey)
    redata = session.post('https://www.coinbig.com/api/publics/v1/order_info', data=newdata).content.decode(encoding='UTF8')
    try:
        redatadict = json.loads(redata)
    except:
        redatadict = 1
        print("获取订单详情getOrderInfo接口信息错误！\n\n",redata)
    return redatadict

In [ ]:
# 接口3：撤销订单
def cancelOrder(session, apiKey, secretKey, orderId):
    Time = int(time.time() * 1000)
    data = {"apikey": apiKey,
            "order_id": orderId, 
            "time": Time}
    newdata = addSign(data, secretKey)
    redata = session.post('https://www.coinbig.com/api/publics/v1/cancel_order', data=newdata).content.decode(encoding='UTF8')
    try:
        redatadict = json.loads(redata)
    except:
        redatadict = 1
        print("撤销订单cancelOrder接口信息错误！\n\n",redata)
    return redatadict

In [ ]:
# 接口4：获取行情
def getTicker(session, symbol):
    data = {"symbol": symbol}
    urljoin = ""
    for key in data.keys():
        urljoin += ("&"+key+"="+str(data[key]))
    urljoin = urljoin[1:]
    redata = session.get('https://www.coinbig.com/api/publics/v1/ticker?'+urljoin).content.decode(encoding='UTF8')
    try:
        redatadict = json.loads(redata)
    except:
        redatadict = 1
        print("获取行情getTicker接口信息错误！\n\n",redata)
    return redatadict

In [ ]:
# 接口5：下单交易(单个挂单)
def trade(session, apiKey, secretKey, tradeType, tradePrice, tradeAmount, symbol):
    Time = int(time.time() * 1000)
    data = {"amount": tradeAmount, 
            "apikey": apiKey, 
            "price": tradePrice, 
            "symbol": symbol, 
            "time": Time, 
            "type": tradeType}
    newdata = addSign(data, secretKey)
    redata = session.post('https://www.coinbig.com/api/publics/v1/trade', data=newdata).content.decode(encoding='UTF8')
    try:
        redatadict = json.loads(redata)
    except:
        redatadict = 1
        print("下单交易trade接口信息错误！\n\n",redata)
    return redatadict

In [ ]:
# 接口6：获取交易深度列表
def getDepthList(session, symbol):
    data = {"symbol": symbol}
    urljoin = ""
    for key in data.keys():
        urljoin += ("&"+key+"="+str(data[key]))
    urljoin = urljoin[1:]
    redata = session.get('https://www.coinbig.com/api/publics/v1/depthList?'+urljoin).content.decode(encoding='UTF8')
    try:
        redatadict = json.loads(redata)
    except:
        redatadict = 1
        print("获取交易深度列表getDepthList接口信息错误！\n\n",redata)
    return redatadict

## 自动交易挖矿程序实现

#### 1.luPrice()价格设置及检查函数，及子函数getCbMarketInfo()，getAcctAsset()，setupCheck()

In [ ]:
def getCbMarketInfo(session, run, symbol, timeDelay):
    # ticker接口获取最新成交信息
    tickerdata = getTicker(session, symbol)
    if tickerdata==1:
        run = 1
        print("\n\n延迟30秒后继续！")
        time.sleep(30)
    else:
        try :
            if int(tickerdata.get("code"))==0:
                tickPrice = tickerdata.get("data").get('ticker')
                # 最新成交价
                tickLastPrice = float(tickPrice.get('last'))
            else:
                print("获取行情失败 | 错误："+tickerdata.get("msg"))
                input("错误确认修复后请按任意键继续")
                run = 1
        except:
            run = 1
            print("页面获取错误，延迟60秒后继续！")
            time.sleep(60)
    time.sleep(timeDelay)
    marketdepthdata = getDepthList(session, symbol)
    if marketdepthdata==1:
        run = 1
        print("\n\n延迟30秒后继续！")
        time.sleep(30)
    else:
        if int(marketdepthdata.get('code'))==0:
            # 取买一和卖一的均价为当前挂单价格，保留小数点后四位
            tickSBPrice = round((float(marketdepthdata.get("data").get("asks")[0].get('price'))+
                                 float(marketdepthdata.get("data").get("bids")[0].get('price')))/2, 4)
        else:
            print("获取行情失败 | 错误："+marketdepthdata.get("msg"))
            input("错误确认修复后请按任意键继续")
            run = 1
    time.sleep(timeDelay)
    return run, tickLastPrice, tickSBPrice

In [ ]:
def getAcctAsset(session, run, apiKey, secretKey, timeDelay):
    assetinfo = getAssetInfo(session, apiKey, secretKey)
    if assetinfo==1:
        run = 1
        print("\n\n延迟30秒后继续！")
        time.sleep(30)
    else:
        if int(assetinfo.get("code"))==0:
            # 获取当前账户总资产
            beginAsset = float(assetinfo.get("data").get("info").get("asset").get("total"))
            # 获取当前账户现金（USDT）
            beginUsdtAsset = float(assetinfo.get("data").get("info").get("free").get("USDT"))
        else:
            print("获取账户资产信息 | 错误："+assetinfo.get("msg"))
            input("错误确认修复后请按任意键继续")
            run = 1
    time.sleep(timeDelay)
    return run, beginAsset, beginUsdtAsset

In [ ]:
def setupCheck(run, lowerPrice, upperPrice, orderAmount, tickLastPrice, tickSBPrice, beginUsdtAsset, beginAsset, fxhPrice):
    if lowerPrice>=upperPrice:
        print("警告！自动交易价格上下限设置错误")
        run = 1
    elif abs(upperPrice-lowerPrice)>0.001*(upperPrice+lowerPrice)/2:
        isCont = input("警告！当前上下限价差超过了千一,矿损较大，继续执行请输入“YES”：")
        if isCont!="YES":
            run = 1
    if abs(tickLastPrice-tickSBPrice)>0.001*tickLastPrice and run==0:
        isCont = input("警告！当前上下限均价和CB盘口均价差距过大，继续执行请输入“YES”：")
        if isCont!="YES":
            run = 1
    if abs((lowerPrice+upperPrice)/2-tickSBPrice)>0.001*tickSBPrice and run==0:
        isCont = input("警告！当前上下限均价和CB现价差距过大，成交率会降低，继续执行请输入“YES”：")
        if isCont!="YES":
            run = 1
    if abs((lowerPrice+upperPrice)/2-fxhPrice)>0.02*fxhPrice and run==0:
        isCont = input("警告！当前上下限均价和非小号行情价差距过大，继续执行请输入“YES”：")
        if isCont!="YES":
            run = 1
    # 检查账户资产是否满足开单条件
    if (orderDirList[orderDirCode]=="buy" and orderAmount*upperPrice>float(beginUsdtAsset)) or \
    (orderDirList[orderDirCode]=="sell" and orderAmount*upperPrice>(float(beginAsset)-float(beginUsdtAsset))):
        print("当前账号余额不足，无法开单，请调低单次挂单量！")
        run = 1
    elif orderAmount*upperPrice>beginAsset*0.8:
        isCont = input("警告！单次挂单数量过大，突发矿损不可控，继续执行请输入“YES”：")
        if isCont!="YES":
            run = 1
    return run

In [ ]:
def luPrice(symbol, apiKey, secretKey, timeDelay):
    
    # 运行通过后开始自动交易，否则无限循环，直到设置符合要求（风控）
    run = 0
    
    while run==0:
        with requests.session() as MKsession:
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}
            MKsession.headers.update(headers)
            
            # 获取非小号市价
            fxhPrice = getFxhPrice(symbol)
            
            # 获取CB最新成交有关信息
            run ,tickLastPrice, tickSBPrice = getCbMarketInfo(MKsession, run, symbol, timeDelay)
            
            # 获取账户资产信息
            if run==0:
                run, beginAsset, beginUsdtAsset = getAcctAsset(MKsession, run, apiKey, secretKey, timeDelay)
                
            if run==0:            
                print("------------------------------------\n*资产*\nUSDT  ："+str(beginUsdtAsset)+"\n总资产："+str(beginAsset)
                      +"\n*行情*\n当前非小号价格   ："+str(fxhPrice)+" USDT\nCoinBig最新成交价："+str(tickLastPrice)+
                      " USDT\nCoinBig买卖均价  ："+str(tickSBPrice)+" USDT\n建议最大开单区间【"+str(round((tickSBPrice+tickLastPrice)*0.5*0.9997, 4))+
                      ","+str(round((tickSBPrice+tickLastPrice)*0.5*1.0003, 4))+"】\n------------------------------------")
                lowerPrice = float(input("请输入自动交易价格下限："))
                upperPrice = float(input("请输入自动交易价格上限："))
                orderAmount = float(input("请输入单次挂单数量"))
                
                # 对设置的lowerPrice，upperPrice，orderAmount进行检查
                run = setupCheck(run, lowerPrice, upperPrice, orderAmount, tickLastPrice, tickSBPrice, beginUsdtAsset, beginAsset, fxhPrice)
                
        # 循环调整, run=0则运行正常，置1跳出，进入交易部分
        if run == 0:
            run = 1
        else:
            run = 0
    return lowerPrice, upperPrice, orderAmount

#### 2.orderProgram()下单函数，及子函数checkAsset()，getOrderPrice()，orderScan()

In [ ]:
def checkAsset(session, run, apiKey, secretKey, timeDelay):
    realassetinfo = getAssetInfo(session, apiKey, secretKey)
    if realassetinfo==1:
        run = 1
    else:
        if int(realassetinfo.get("code"))==0:
            realAsset = float(realassetinfo.get("data").get("info").get("asset").get("total"))
            # 当前资产小于初始资产的0.97倍（可修改）时报警退出！
            if realAsset<beginAsset*0.97:
                print("账号资产波动太大，请检查后重新运行！")
                writeOrderInfo("账号资产波动太大，请检查后重新运行！")
                run = 1
        else:
            print("获取账户资产信息 | 错误："+realassetinfo.get("msg"))
            run = 1
    time.sleep(timeDelay)
    return run

In [ ]:
def getOrderPrice(MODE, session, run, symbol, upperPrice, lowerPrice, timeDelay):
    if MODE == 0:
        # depthdata获取当前挂单情况，"bids"为买，"asks"为卖
        depthdata = getDepthList(session, symbol)
        if depthdata==1:
            run = 1
        else:
            if int(depthdata.get('code'))==0:
                # 取买一和卖一的均价为当前挂单价格，保留小数点后四位
                orderPrice = round((float(depthdata.get("data").get("asks")[0].get('price'))+
                                   float(depthdata.get("data").get("bids")[0].get('price')))/2, 4)
            else:
                print("获取市场挂单信息失败 | 错误："+depthdata.get("msg"))
                run = 1
        time.sleep(timeDelay)
    elif MODE==1:
        if orderDirCode==0:
            # 买入价格为设置价格上限
            orderPrice = upperPrice
        elif orderDirCode == 1:
            # 卖出价格为设置价格下限
            orderPrice = lowerPrice
    return run, orderPrice

In [ ]:
def orderScan(session, run, apiKey, secretKey, orderId, orderDirCode, orderAmount, beginOrderAmount, timeDelay):
    orderScanCounter = 0
    while 1:
        orderinfo = getOrderInfo(session, apiKey, secretKey, orderId)
        if orderinfo==1:
            run = 1
            break
        else:
            if int(orderinfo.get("code"))==0:
                if int(orderinfo.get("data").get("orderinfo").get("status"))==1:
                    orderLeftCount = orderAmount
                    print("订单"+str(orderId)+"暂无成交")
                elif int(orderinfo.get("data").get("orderinfo").get("status"))==2:
                    orderLeftCount = float(orderinfo.get("data").get("orderinfo").get("leftCount"))
                    print("订单"+str(orderId)+"尚未成交数量："+str(orderLeftCount))
                elif int(orderinfo.get("data").get("orderinfo").get("status"))==3:
                    print("订单"+str(orderId)+"[价格："+str(orderPrice)+"，数量："+str(orderAmount)+"]完全成交！")
                    writeOrderInfo("订单"+str(orderId)+"完全成交！\n------------------------------------")
                     # 订单完成，改变交易方向,初始化挂单量
                    orderDirCode = (orderDirCode+1)%2
                    orderAmount = beginOrderAmount
                    # 退出监控
                    break
            else:
                print("获取订单信息失败 | 错误："+orderinfo.get("msg"))
                writeOrderInfo("获取订单信息失败 | 错误："+orderinfo.get("msg")+"\n")
                run = 1
                break

        #长时间未成交撤销后，重按最新价挂单
        orderScanCounter +=1
        if orderScanCounter>15:
            print("订单长时间未完全成交(未成交数量："+str(orderLeftCount)+")，即将撤销重挂！")
            writeOrderInfo("订单长时间未完全成交(未成交数量："+str(orderLeftCount)+")，撤销重挂！")
            time.sleep(timeDelay)
            cancelorderdata = cancelOrder(session, apiKey, secretKey, orderId)
            if cancelorderdata==1:
                run = 1
                break
            else:   
                if int(cancelorderdata.get("code"))==0:
                    print("撤单成功\n------------------------------------")
                    writeOrderInfo("订单"+str(orderId)+"撤单成功\n")
                    # 撤单成功后按未成交数量和方向（不改动）继续挂单
                    orderAmount = round(orderLeftCount, 4)
                    #退出监控
                    break
                else:
                    print("撤单失败 | 错误："+cancelorderdata.get("msg"))
                    writeOrderInfo("撤单失败 | 错误："+cancelorderdata.get("msg")+"\n")
                    run = 1
                    break
    return run, orderDirCode, orderAmount

In [ ]:
def orderProgram(MODE, symbol, orderDirList, orderDirCode, lowerPrice, upperPrice, orderAmount, apiKey, secretKey, timeDelay):
    beginOrderAmount = orderAmount
    with requests.session() as CBsession:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}
        CBsession.headers.update(headers)
        run = 0
        counter = 0
        while run==0:
            # 检测当前资产和起始资产相比是否波动太大
            run = checkAsset(CBsession, run, apiKey, secretKey, timeDelay)
            if run==1:
                break
            # 获取下单价
            run, orderPrice = getOrderPrice(MODE, CBsession, run, symbol, upperPrice, lowerPrice, timeDelay)
            if run==1:
                break
            # 当前价格满足则挂单(限价单)
            orderDir = orderDirList[orderDirCode]
            if (MODE==0 and lowerPrice<=orderPrice<=upperPrice) or MODE==1:
                writeOrderInfo("【挂单】 方向："+orderDir+" | 价格："+str(orderPrice)+" | 数量："+str(orderAmount)+
                               "       Time:"+str(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime())))
                # 下单
                tradedata = trade(CBsession, apiKey, secretKey, orderDir, orderPrice, orderAmount, symbol)
                if tradedata==1:
                    break
                else:
                    if int(tradedata.get("code"))==0:
                        counter = 0
                        orderErrCounter = 0
                        print("[方向："+orderDir+"|价格："+str(orderPrice)+"|数量："+str(orderAmount)+"]挂单成功!")
                        writeOrderInfo("挂单成功")
                        orderId = tradedata.get("data").get("order_id")
                        time.sleep(timeDelay)
                        # 下单后循环监控该订单情况，完全成交后才继续
                        run, orderDirCode, orderAmount = orderScan(CBsession, run, apiKey, secretKey, orderId, orderDirCode, orderAmount, beginOrderAmount, timeDelay)
                        if run==1:
                            break 
                    else:
                        orderErrCounter+=1
                        if orderErrCount>4:
                            print("多次挂单失败,自动退出 | 错误："+tradedata.get("msg"))
                            writeOrderInfo("多次挂单失败 | 错误："+tradedata.get("msg"))
                            run = 1
                            break
                        else:
                            print(str(orderDir)+orderDir+"挂单失败,尝试第"+str(orderErrCount)+"次 | 错误："+tradedata.get("msg"))
                            writeOrderInfo(str(orderDir)+orderDir+"挂单失败,尝试第"+str(orderErrCount)+"次 | 错误："+tradedata.get("msg"))
            else:
                counter+=1
                if counter>80:
                    print("当前市场价长时间不符合开单区间，请修改开单价区间或稍后继续！")
                    break
                else:
                    print("当前价格["+str(orderPrice)+"]不满足开单条件，继续监控")

#### 3.主函数

In [ ]:
if __name__ == "__main__":
    """
    MODE为模式设置，有0和1两种，设为0时，下单价格为CB最新成交价，设为1时，下单价格为自定义的上下限价格。
    （MODE=1能提高订单成交速度，但矿损会扩大，选择此方式的话请谨慎选择上下限价格）
    """
    MODE = 0
    
    # API密钥配置
    apiKey = "***"
    secretKey = "***"

    # 设置交易品种，0为BTC，1为ETH
    symbolList = ["BTC_USDT", "ETH_USDT"]
    symbolCode = 1
    symbol = symbolList[symbolCode]

    # 设置交易方向，orderDirList全为“buy”则一直挂买单，全为“sell”则一直挂卖单
    orderDirList = ["buy", "sell"]
    # 0则先买后卖，1则先卖后买
    orderDirCode = 0
    
    # 全局时间延长设置（单位：s），在网络不稳定或平台有API访问时间限制时进行调整
    timeDelay = 0
    
    # 价格设置及检查
    lowerPrice, upperPrice, orderAmount = luPrice(symbol, apiKey, secretKey, timeDelay)
    
    # 下单主程序
    orderProgram(MODE, symbol, orderDirList, orderDirCode, lowerPrice, upperPrice, orderAmount, apiKey, secretKey, timeDelay)
                    
    input("程序结束，请输入任意字符关闭！")